# 针对建筑规范的清洗

##### 1.先pdf转word，然后word转txt，能够很好保留制表符\t
##### 2.pdf直接提取txt，可以获得表明，但是会失去制表符号

In [ ]:
with open('test.txt', 'r', encoding='utf-8') as f:
    #data = f.readlines()
    data2 = f.read() #因为文件已经打开了，不能再打开一次，所以这条指令没用
    print(data2)

#找出所有表格名称
import re
#rule = '^续?^表(\s*\d+\s*\.){2}\s*\d+\s*-?(?<=-)\s*\d+?\s+\w+?\s*?\n$'
#print(re.findall(rule, data2))
#for line in data:
    #if re.match('^(续?表)(\s*\d+\s*)((\.\s*\d+\s*){2}|\s*).*?', line):print(line)
results = re.findall('续?表[^\S\r\n]*\d+\s*\.?\s*\.*\s*\d*\s*\.*\d*[^\S\r\n]*\-*[^\S\r\n]*\d*[^\S\r\n]+[^。，,；所规和\n\s]{6,}|续表.*', data2)
if len(results)>0:
    print(f'找到{len(results)}个')
    print('\n'.join(results))
#results2 = re.findall('续表.*', data2)
#if len(results2)>0:
    #print('\n'.join(results2))

In [ ]:
#找出所有表格
import re
with open('test3.txt', 'r', encoding='utf8') as f:
    data3 = f.read()
result = re.findall('\n\s*\d+\.\d+\.\d+\s*\n*[\s\S]*?\s*(?=\n\s*\d+\.\d+\s)', data3)
print(len(result))
tables = []
for i, r in enumerate(result):
    if r != '':
        print(f'<第{i}段>')
        r = re.sub("\t", '|', r)  #显示制表符
        r = re.sub('\n+\s*', '\n', r)  #多个换行符变一个换行符,且删除行前空白
        r = re.sub('(?<=\d)[ \t]+(?=\w)', ' ', r)  #将数字与文字之间的空白变成一个空格。
        r = re.sub('(?<=\w)[ \t]+(?=\w)', ' ', r)  #将文字之间的空白变成一个空格
        r = re.sub('。\s+', '。\n', r)  #句号空白变成句号回车
        r = re.sub('(?<=[\u4e00-\u9fa5])[ \t](?=[\u4e00-\u9fa5])', '', r)  #删除中文之间的非换行空白
        r = re.sub('(?<=\d)\)\n', ')', r)  #)后面不用回车
        r = re.sub('(?<=[图第，])\n(?=\d)', '', r)  #图 数字不用回车
        if '表' in r:
            #字符中有表字时，以表或注分割
            sp = re.split('\n+(?=[表注])', r)
            for j, s in enumerate(sp):
                print(f'\n<第{i}段><第{j}切片>')
                if '注' or '表' in s:
                    #如果切片中有注或表
                    s = re.split('\n(?=\d+\.)', s)
                    for k, ss in enumerate(s):
                        if not re.match('^表', ss):
                            #非表格的内容要去除两个中文字之间的换行
                            ss = re.sub('(?<=[\u4e00-\u9fa5、])\n(?=[\u4e00-\u9fa5])', '', ss) #去除两个中文字之间的换行
                        else:tables.append(ss)
                        if len(ss)>0:
                            print(f'\n<第{i}段><第{j}切片>之{k}')
                            print(ss+'\n')
                else:
                    print(s)
        else:
            r = re.sub('(?<=[\u4e00-\u9fa5、])\n(?=[\u4e00-\u9fa5])', '', r) #去除两个中文字之间的换行
            print(r)

#### 下一步研究的就是表头的合并

In [12]:
import pandas as pd
t = tables[0]
t = re.sub('(?<=结构体系\|)\n','', t)
t = re.sub('\|', ',', t)
#print(rows)
'''
步骤：
1.不计标题，如果第一行第一个不空，后面连续n行的第一个为空，即表头n+1行
2.如果第一行的第一列不空，后面连续n列第一个为空，即表头为n+1列
3.对行和列的表头进行合并
'''
df = pd.read_csv(StringIO(t))
df

<>:3: SyntaxWarning: invalid escape sequence '\|'
<>:4: SyntaxWarning: invalid escape sequence '\|'
<>:3: SyntaxWarning: invalid escape sequence '\|'
<>:4: SyntaxWarning: invalid escape sequence '\|'
C:\Users\ZONETEAM\AppData\Local\Temp\ipykernel_38124\1381839521.py:3: SyntaxWarning: invalid escape sequence '\|'
  t = re.sub('(?<=结构体系\|)\n','', t)
C:\Users\ZONETEAM\AppData\Local\Temp\ipykernel_38124\1381839521.py:4: SyntaxWarning: invalid escape sequence '\|'
  t = re.sub('\|', ',', t)
C:\Users\ZONETEAM\AppData\Local\Temp\ipykernel_38124\1381839521.py:3: SyntaxWarning: invalid escape sequence '\|'
  t = re.sub('(?<=结构体系\|)\n','', t)
C:\Users\ZONETEAM\AppData\Local\Temp\ipykernel_38124\1381839521.py:4: SyntaxWarning: invalid escape sequence '\|'
  t = re.sub('\|', ',', t)


NameError: name 'StringIO' is not defined

In [56]:
#找出表格的另一种方式
import re
with open('test3.txt', 'r', encoding='utf8') as f:
    data4 = f.read()
result4 = re.split('\n(?=\d)(', data4)
print(len(result4))
for i, r in enumerate(result4):
    if r != '':
        print(f'<第{i}段>')
        print(r)

1605
<第0段>

中华人民共和国住房和城乡建设部 公   告

第788号
　


关于发布行业标准
《高层建筑混凝土结构技术规程》的公告

　　　现批准《高层建筑混凝土结构技术规程》为行业标准，编号 为JGJ3-2010,     自2011年10月1日起实施。其中，第3.8.1、
<第1段>
3.9.1 、3.9.3 、3.9.4 、4.2.2 、4.3.1 、4.3.2 、4.3.12、
<第2段>
4.3.16 、5.4.4 、5.6.1 、5.6.2 、5.6.3 、5.6.4 、6.1.6 、6.3.2、
<第3段>
6.4.3 、7.2.17 、8.1.5 、8.2.1 、9.2.3 、9.3.7 、10.1.2、
<第4段>
10.2.7 、10.2.10 、10.2.19 、10.3.3 、10.4.4 、10.5.2、
　10.5.6、11.1.4.条为强制性条文，必须严格执行。原行业标准 《高层建筑混凝土结构技术规程》JGJ3-2002   同时废止。
　　　本规程由我部标准定额研究所组织中国建筑工业出版社出版 发行。

中华人民共和国住房和城乡建设部 2010年10月21日







目   次

<第5段>
1     总 则 	1
<第6段>
2    术语和符号 	2
<第7段>
2.1    术 语 	2
<第8段>
2.2    符 号 	4
<第9段>
3    结构设计基本规定 	 7
<第10段>
3.1    一般规定 	7
<第11段>
3.2    材料 	8
<第12段>
3.3    房屋适用高度和高宽比 	9
<第13段>
3.4    结构平面布置 	 11
<第14段>
3.5    结构竖向布置 	 14
<第15段>
3.6    楼盖结构 	16
<第16段>
3.7    水平位移限值和舒适度要求 	 17
<第17段>
3.8    构件承载力设计 	 20
<第18段>
3.9    抗震等级 	 21
<第19段>
3.10    特一级构件设计规定 	23
<第20段>
3.11    结构抗震性能设计 	25
<第21段>
3.12.   抗连续倒塌设计基本要求 	28
<第22段>
4    荷载和地震作用 	31
<第23段>
4.1    竖向荷载 	31
<第24段>
4.2

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\ZONETEAM\AppData\Local\Temp\ipykernel_4132\4218818978.py:5: SyntaxWarning: invalid escape sequence '\d'
  result4 = re.split('\n(?=\d)', data4)
